In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### Data Collection

In [33]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/module3/exercise/module3_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/module3/exercise/module3_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module3_exercise_train.csv')
download_file(test_data_url, 'module3_exercise_test.csv')

Downloaded module3_exercise_train.csv from https://www.raphaelcousin.com/modules/module3/exercise/module3_exercise_train.csv
Downloaded module3_exercise_test.csv from https://www.raphaelcousin.com/modules/module3/exercise/module3_exercise_test.csv


In [34]:
df_train =  pd.read_csv("module3_exercise_train.csv", sep=",", index_col='id')

In [35]:
df_train

,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
id,,,,,,,,,,,,,,,,,
254,2,1,4.0,1,1977.0,two,440,0,55,0,0,165,0,0,7,2010.0,127500
1066,1,1,5.0,1,1983.0,two,612,349,40,0,0,0,0,0,9,2009.0,316600
638,4,1,10.0,1,1998.0,two,420,144,123,0,0,0,0,0,7,2006.0,258000
799,3,1,8.0,0,1916.0,one,180,0,0,0,140,0,0,0,8,2009.0,135000
380,2,1,5.0,0,2005.0,two,438,108,0,0,0,0,0,0,3,2006.0,167240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,4,1,7.0,0,-1.0,zero,0,0,192,0,0,0,0,0,11,2008.0,130000
1130,3,1,6.0,0,1964.0,two,504,0,0,0,0,0,0,0,7,2008.0,145000
1294,3,1,7.0,1,1996.0,three,889,220,0,0,0,0,0,0,7,2009.0,265000


### Data Preprocessing

In [36]:
def minimal_data_prep(data_df):
    data_df.drop('GarageCars', axis=1, inplace=True)
    data_df.fillna(-1, inplace=True)
    return data_df

In [37]:
df_train=minimal_data_prep(df_train)

### Model Building and Evaluation

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [39]:
# Preparing the data
y = df_train['SalePrice']
X = df_train[["BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces", "GarageYrBlt", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal", "MoSold", "YrSold"]]

In [40]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Initialize and train the Linear Regression model on the training set
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [42]:
# Function for evaluating the model. Here with MAE :

def pred_eval(model, X_data, y_target):
    y_pred = model.predict(X_data)
    mae=mean_absolute_error(y_pred,y_target)
    return (y_pred,mae)

In [43]:
# Evaluate the trained model on the validation set :
y_pred,mae = pred_eval(model, X_val, y_val)
print('MAE :', mae)

MAE : 32533.8389835379


### Generating Submission File

In [44]:
df_test =  pd.read_csv("module3_exercise_test.csv", sep=",", index_col='id')

In [45]:
# Preparing the data
df_test = minimal_data_prep(df_test)
X_test = df_test # df_test does not have the 'SalePrice' column 

In [47]:
# Train on all information you have
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [48]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [53]:
submission = pd.DataFrame({
    'id': X_test.index,
    'SalePrice': y_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

,id,SalePrice
0,892,269802.561259
1,1105,234199.005970
2,413,262193.455621
3,522,223122.978415
4,1036,150845.571501
